In [ ]:

import matplotlib.pyplot as plt
import opensoundscape
import glob
import os
import pandas as pd
import numpy as np
import sklearn
import librosa
import torch
import wandb
import random
from  apply_transfer_function import TransferFunction
from convert_audio_to_bits import convert_audio_to_bits

In [ ]:
# load model
model = opensoundscape.ml.cnn.load_model('/home/michaela/CV4E/model_states/best.model')

In [ ]:
# load training data 
train_clips = pd.read_csv('/home/michaela/CV4E/labeled_data/train_balanced_1500.csv', index_col=[0,1,2]) 

In [ ]:
train_scores = model.predict(train_clips, num_workers=16,batch_size=128) 

In [ ]:
train_scores.columns = ['pred_D','pred_A','pred_B']
train_all = train_clips.join(train_scores)
train_evaluation = train_all.reset_index()
# train_scores_normal = train_scores.reset_index()
# train_clips_normal = train_clips.reset_index()

In [ ]:
tf_path = '/mnt/ssd-cluster/michaela/data/transfer_functions/688_130415_B_HARP_DCPP01A.tf'
TF = pd.read_csv(tf_path,delim_whitespace=True,header=None)
TF.columns=['frequency','calibration']

In [ ]:
# train_clips_normal['end_time']= (train_clips_normal['end_time']).astype(int)
# train_clips_normal['start_time']= (train_clips_normal['start_time']).astype(int)

# train_scores_normal['end_time']= (train_scores_normal['end_time']).astype(int)
# train_scores_normal['start_time']= (train_scores_normal['start_time']).astype(int)
# train_evaluation = pd.merge(train_scores_normal, train_clips_normal, on =['file','start_time','end_time'])

In [ ]:
# D call train
D_eval_index = train_evaluation.index[train_evaluation['D']==1]
D_eval = train_evaluation.loc[D_eval_index]
D_noise_index = train_evaluation.index[train_evaluation['D']==0]
D_noise = train_evaluation.loc[D_noise_index]

plt.hist(D_noise['pred_D'],bins=40,edgecolor='black',alpha=0.5,color='blue',label='Noise prediction score')
plt.hist(D_eval['pred_D'],bins=40,edgecolor='black',alpha=0.5,color='orange',label='D call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('D call prediction scores train')
plt.legend(loc='upper right') # look at low scoring D calls and high scoring negatives! 

In [ ]:
# investigate false postives for D call train
# threshold of zero for these (before sigmoid function applyed to activation layer)
# D_noise
# D_noise_fp = D_noise[D_noise['pred_D'] > -0]
# D_noise_fp = D_noise_fp.reset_index()

In [ ]:
tf_path = '/mnt/ssd-cluster/michaela/data/transfer_functions/688_130415_B_HARP_DCPP01A.tf'
TF = pd.read_csv(tf_path,delim_whitespace=True,header=None)
TF.columns=['frequency','calibration']

In [ ]:
# evaluate false positives in training data (D calls)
# D_fp =opensoundscape.Audio.from_file(D_noise_fp['file'][39],sample_rate=3200,offset=D_noise_fp['start_time'][39],duration=15) # is this the first 30 sec?
# bits = 16 
# abs_max = 2 ** (bits - 1) 
# D_fp.samples = np.float64(D_fp.samples)*abs_max
# D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming',window_samples=1600,overlap_samples=1400,fft_size=3200,decibel_limits=(-200,200),scaling='density')
# D_fp1_TF = apply_transfer_function(D_fp1,TF,decibel_limits=(40,140))
# D_fp1_TF.bandpass(10,150).to_image() 

In [ ]:
# evaluate D call false postives with high prediction scores
# shocker they're all D calls that were incorrectly labeled. 

D_noise_fp = D_noise[D_noise['pred_D'] > 0]
D_noise_fp = D_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive
for index, row in D_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')


In [ ]:
# evaluate D call true postives with low prediction scores
# shocker they're all D calls that were incorrectly labeled. 

D_eval_tp = D_eval[D_eval['pred_D'] < 0]
D_eval_tp = D_eval_tp.reset_index()

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive
for index, row in D_eval_tp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    
    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=sample_rate, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()
    
    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

In [ ]:
# A call train
A_eval_index = train_evaluation.index[train_evaluation['A NE Pacific']==1]
A_eval = train_evaluation.loc[A_eval_index]
A_noise_index = train_evaluation.index[train_evaluation['A NE Pacific']==0]
A_noise = train_evaluation.loc[A_noise_index]

plt.hist(A_noise['pred_A'],edgecolor='black',bins=40,alpha=0.5,color='blue',label='Noise prediction score')
plt.hist(A_eval['pred_A'],edgecolor='black',bins=40,alpha=0.5,color='orange',label='A call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('A call prediction scores train')
plt.legend(loc='upper right') 

In [ ]:
A_noise_fp = A_noise[A_noise['pred_A'] > -17]
A_noise_fp = A_noise_fp.reset_index(drop=True) # Cant plot too many cause its slowwwww

In [ ]:
len(A_noise_fp)

In [ ]:
# evaluate A call false postives with high prediction scores
# Looking at false positve A calls with a score greater than -20 on infite scale
A_noise_fp = A_noise[A_noise['pred_A'] > -17]
A_noise_fp = A_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive
for index, row in A_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')


In [ ]:
# B call train
B_eval_index = train_evaluation.index[train_evaluation['B NE Pacific']==1]
B_eval = train_evaluation.loc[B_eval_index]
B_noise_index = train_evaluation.index[train_evaluation['B NE Pacific']==0]
B_noise = train_evaluation.loc[B_noise_index]

plt.hist(B_noise['pred_B'],bins=40,edgecolor='black',alpha=0.5,color='blue',label='Noise prediction score')
plt.hist(B_eval['pred_B'],bins=40,edgecolor='black',alpha=0.5,color='orange',label='B call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('B call prediction scores train')
plt.legend(loc='upper right')

In [ ]:
# evaluate B call false postives with high prediction scores
# Looking at false positve A calls with a score greater than -20 on infite scale
B_noise_fp = B_noise[B_noise['pred_B'] > -20]
B_noise_fp = B_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive
for index, row in B_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# B_noise=opensoundscape.Audio.from_file(B_noise.iloc[0]['file'],sample_rate=2000,offset=B_noise['start_time'][0],duration=15) # is this the first 30 sec?

# bits = 16 
# abs_max = 2 ** (bits - 1)
# B_noise.samples = np.float64(B_noise.samples)*abs_max

# #test_spec = Spectrogram.from_audio(test_wav, window_type='hamming',window_samples=1000,overlap_samples=900,fft_size=2000,decibel_limits=(-200,200),scaling='density')
# test_spec = opensoundscape.Spectrogram.from_audio(B_noise, window_type='hamming',window_samples=1000,overlap_samples=900,fft_size=2000,decibel_limits=(-200,200),scaling='density')

In [ ]:
# VALIDATE 
# run model on validation dataset
validate_clips = pd.read_csv('/home/michaela/CV4E/labeled_data/validate_clips.csv', index_col=[0,1,2])
validate_scores = model.predict(validate_clips, num_workers=16,batch_size=128)

In [ ]:
validate_scores.columns = ['pred_D','pred_A','pred_B']
validate_all = validate_clips.join(validate_scores)
validate_evaluation = validate_all.reset_index()

In [ ]:
# D call validate
D_eval_index = validate_evaluation.index[validate_evaluation['D']==1]
D_eval = validate_evaluation.loc[D_eval_index]
D_noise_index = validate_evaluation.index[validate_evaluation['D']==0]
D_noise = validate_evaluation.loc[D_noise_index]

plt.hist(D_noise['pred_D'],bins=40,alpha=0.5,edgecolor='black',color='blue',label='Noise prediction score')
plt.hist(D_eval['pred_D'],bins=40,alpha=0.5,edgecolor='black',color='orange',label='D call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('D call prediction scores validate')
plt.legend(loc='upper right')

In [ ]:
# evaluate D call false postives with high prediction scores
# Looking at false positve A calls with a score greater than -20 on infite scale
D_noise_fp = D_noise[D_noise['pred_D'] > -3]
D_noise_fp = D_noise_fp.reset_index(drop=True)

In [ ]:
len(D_noise_fp)

In [ ]:
# evaluate D call false postives with high prediction scores
# Looking at false positve D calls with a score greater than 0 on infite scale
D_noise_fp = D_noise[D_noise['pred_D'] > -3]
D_noise_fp = D_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive (HIGH SCORING NEGATIVE)
for index, row in D_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# B call validate
B_eval_index = validate_evaluation.index[validate_evaluation['B NE Pacific']==1]
B_eval = validate_evaluation.loc[B_eval_index]
B_noise_index = validate_evaluation.index[validate_evaluation['B NE Pacific']==0]
B_noise = validate_evaluation.loc[B_noise_index]

plt.hist(B_noise['pred_B'],bins=40,edgecolor='black',alpha=0.5,color='blue',label='Noise prediction score')
plt.hist(B_eval['pred_B'],bins=40,edgecolor='black',alpha=0.5,color='orange',label='B call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('B call prediction scores validate')
plt.legend(loc='upper right')

In [ ]:
B_noise_fp = B_noise[B_noise['pred_B'] > -20]
B_noise_fp = B_noise_fp.reset_index(drop=True)

In [ ]:
len(B_noise_fp)

In [ ]:
# evaluate B call false postives with high prediction scores
# Looking at false positve B calls with a score greater than -20 on infinite scale
B_noise_fp = B_noise[B_noise['pred_B'] > -20]
B_noise_fp = B_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive (HIGH SCORING NEGATIVE)
for index, row in B_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# A call validate
A_eval_index = validate_evaluation.index[validate_evaluation['A NE Pacific']==1]
A_eval = validate_evaluation.loc[A_eval_index]
A_noise_index = validate_evaluation.index[validate_evaluation['A NE Pacific']==0]
A_noise = validate_evaluation.loc[A_noise_index]

plt.hist(A_noise['pred_A'],bins=40,edgecolor='black',alpha=0.5,color='blue',label='Noise prediction score')
plt.hist(A_eval['pred_A'],bins=40,edgecolor='black',alpha=0.5,color='orange',label='A call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('A call prediction scores validate')
plt.legend(loc='upper right')

In [ ]:
A_noise_fp = A_noise[A_noise['pred_A'] > -20]
A_noise_fp = A_noise_fp.reset_index(drop=True)

In [ ]:
len(A_noise_fp)

In [ ]:
# evaluate A call false postives with high prediction scores
# Looking at false positve A calls with a score greater than -20 on infinite scale
A_noise_fp = A_noise[A_noise['pred_A'] > -20]
A_noise_fp = A_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive (HIGH SCORING NEGATIVE)
for index, row in A_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# TEST Model on CINMS18B ###
test_clips = pd.read_csv('/home/michaela/CV4E/labeled_data/CINMS18B_one_hot_clips.csv', index_col=[0,1,2]) # read in test data
test_clips.sum() 
# model settings for test data
model.preprocessor.pipeline.to_spec.params.window_samples = 1000 # 100 window samples
model.preprocessor.pipeline.to_spec.params.overlap_samples = 900 # 90% overlap, for 2000 Fs this means 900 samples, and 0.05 sec bins
model.preprocessor.pipeline.to_spec.params.fft_size = 2000 # FFT = Fs, 1 Hz bins
test_scores = model.predict(test_clips, num_workers=16,batch_size=128)

In [ ]:
test_scores.columns = ['pred_D','pred_A','pred_B']
test_all = test_clips.join(test_scores)
test_evaluation = test_all.reset_index()

#test_clips_new.iloc[0]['file']
#test_clips_new = test_clips.reset_index()

In [ ]:
model.preprocessor.forward(test_clips.iloc[0].name[0]) # preprocess the sample
# fails when trying to apply the transfer function. seems like its something in the naming scheme...
# going to have to adjust the way I apply my transfer function for the model. I can't give it every single model. Needs to be input in the beginning somehow... 
# or going to have to figure out how to load it as the model runs - in the function. Idk. Wish I had more time. 
# can't just have my nice transfer function saved in my model. need config file. need to make config file. 

In [ ]:
# D call test
D_eval_index = test_evaluation.index[test_evaluation['D']==1]
D_eval = test_evaluation.loc[D_eval_index]
D_noise_index = test_evaluation.index[test_evaluation['D']==0]
D_noise = test_evaluation.loc[D_noise_index]

plt.hist(D_noise['pred_D'],bins=40,alpha=0.5,edgecolor='black',color='blue',label='Noise prediction score')
plt.hist(D_eval['pred_D'],bins=40,alpha=0.5,edgecolor='black',color='orange',label='D call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('D call prediction scores test')
plt.legend(loc='upper right') # this is progress. look at those high scoring missed detections
# blue is all of the examples in the D call column that did not actually contain a D call. 

In [ ]:
# evaluate D call false postives with high prediction scores
# Looking at false positve D calls with a score greater than 0 on infite scale
D_noise_fp = D_noise[D_noise['pred_D'] > 10]
D_noise_fp = D_noise_fp.reset_index(drop=True)

In [ ]:
len(D_noise_fp)

In [ ]:
# evaluate D call false postives with high prediction scores
# Looking at false positve D calls with a score greater than 0 on infite scale
D_noise_fp = D_noise[D_noise['pred_D'] > 10]
D_noise_fp = D_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive (HIGH SCORING NEGATIVE)
for index, row in D_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# B call test
B_eval_index = test_evaluation.index[test_evaluation['B NE Pacific']==1]
B_eval = test_evaluation.loc[B_eval_index]
B_noise_index = test_evaluation.index[test_evaluation['B NE Pacific']==0] 
B_noise = test_evaluation.loc[B_noise_index]

plt.hist(B_noise['pred_B'],bins=40,alpha=0.5,edgecolor='black',color='blue',label='Noise prediction score')
plt.hist(B_eval['pred_B'],bins=40,alpha=0.5,edgecolor='black',color='orange',label='B call prediction score')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.semilogy()
plt.title('B call prediction scores test')
plt.legend(loc='upper right') # Ok this is a start! could be worse... needs more training examples. 

In [ ]:
# evaluate B call false postives with high prediction scores
# Looking at false positve D calls with a score greater than 0 on infite scale
B_noise_fp = B_noise[B_noise['pred_B'] > -15]
B_noise_fp = B_noise_fp.reset_index(drop=True)

In [ ]:
len(B_noise_fp)

In [ ]:
# evaluate D call false postives with high prediction scores
# Looking at false positve D calls with a score greater than 0 on infite scale
B_noise_fp = B_noise[B_noise['pred_B'] > -15]
B_noise_fp = B_noise_fp.reset_index(drop=True)

# bit transform
bits = 16 
abs_max = 2 ** (bits - 1)

# Create a for loop to process each false positive (HIGH SCORING NEGATIVE)
for index, row in B_noise_fp.iterrows():
    file_path = row['file']
    start_time = row['start_time']
    annotations = [row['D'],row['A NE Pacific'], row['B NE Pacific']]
    predictions = [row['pred_D'],row['pred_A'], row['pred_B']]
    rounded_predictions = [round(p, 2) for p in predictions]

    # Load the audio
    D_fp = opensoundscape.Audio.from_file(file_path, sample_rate=3200, offset=start_time, duration=15)
    
    # Scale the audio
    D_fp.samples = np.float64(D_fp.samples) * abs_max
    
    # Create a spectrogram
    D_fp1 = opensoundscape.Spectrogram.from_audio(D_fp, window_type='hamming', window_samples=1600, 
                                                  overlap_samples=1400, fft_size=3200, 
                                                  decibel_limits=(-200,200), scaling='density')
    
    # Apply the transfer function
    D_fp1_TF = apply_transfer_function(D_fp1, TF, decibel_limits=(40, 140))
    
    # Bandpass filter and plot
    # 
    filtered_image = D_fp1_TF.bandpass(10, 150).to_image()

    # Display the image using matplotlib
    plt.imshow(filtered_image)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

    print(f'Annotations: D={annotations[0]},A NE Pacific={annotations[1]}, B NE Pacific={annotations[2]}')
    print(f'Predictions: D={rounded_predictions[0]},A NE Pacific={rounded_predictions[1]}, B NE Pacific={rounded_predictions[2]}')

In [ ]:
# extra/random/backup code 

In [ ]:
len(D_eval_index)

In [ ]:
def apply_transfer_function(spec,tf_dataframe,decibel_limits=None):
    """
    apply transfer function to opensoundscape.Spectrogram object
    
    helper function to apply transfer function to Spectrogram
    transfer function is list of | freq | dB offset |
    we should interpolate to the frequencies contained in the specrogram

    Args:
        spec: a Specrogram object
        tf_dataframe: dataframe with columns 'freq' (frequencies in Hz) and 'intensity' (dB offset)
        decibel_limits: default None will use original spectrogram's .decibel_units attribute;
            optionally specify a new decibel_limits range for the returned Spectrogram
    """
    if decibel_limits is None:
        decibel_limits = spec.decibel_limits
        
    #extract frequency column and intensity column from transfer function dataframe
    transfer_function_freqs = tf_dataframe.frequency.values
    transfer_function_offsets = tf_dataframe.calibration.values
    
    # linearly interpolate the frequencies from the transfer function table
    # onto the frequencies of the spectrogram to get offsets for each spectrogram row
    spec_offsets = np.interp(spec.frequencies,transfer_function_freqs, transfer_function_offsets)
    
    # add the offset values to each row of the spectrogram
    new_spec_values = (spec.spectrogram.transpose() + np.array(spec_offsets)).transpose()
    
    #create a new spectrogram object with the new values
    return opensoundscape.Spectrogram(new_spec_values,times=spec.times,frequencies=spec.frequencies,decibel_limits=decibel_limits)

In [ ]:
25,920*10*20

In [ ]:
validate_A_pos_idx=validate_evaluation.index[validate_evaluation['pred_A']>-16]

In [ ]:
validate_A_pos = validate_evaluation.iloc[validate_A_pos_idx] # v excited to get back to my computer.


In [ ]:
validate_A_pos 

In [ ]:
B_noise=opensoundscape.Audio.from_file(B_noise.iloc[0]['file'],sample_rate=2000,offset=B_noise['start_time'][0],duration=15) # is this the first 30 sec?

bits = 16 
abs_max = 2 ** (bits - 1)
B_noise.samples = np.float64(B_noise.samples)*abs_max

#test_spec = Spectrogram.from_audio(test_wav, window_type='hamming',window_samples=1000,overlap_samples=900,fft_size=2000,decibel_limits=(-200,200),scaling='density')
test_spec = opensoundscape.Spectrogram.from_audio(B_noise, window_type='hamming',window_samples=1000,overlap_samples=900,fft_size=2000,decibel_limits=(-200,200),scaling='density')
new_spec = apply_transfer_function(test_spec,TF,decibel_limits=(40,140))
#new_spec.bandpass(10,150).plot()
new_spec.bandpass(10,150).to_image() # ok so seems to be working fine????

In [ ]:
B_noise

In [ ]:
A_eval_index = validate_evaluation.index[validate_evaluation['A NE Pacific']==1]
validate_A_pos_idx=validate_evaluation.index[validate_evaluation['pred_A']>-16]

In [ ]:
print(len(validate_A_pos_idx))

In [ ]:
print(len(A_eval_index )) 

In [ ]:
samples = model.generate_cams(samples=train_clips, classes='D',guided_backprop=True) # its clearly doing somethnig if its taking 20 minutes
#samples

In [ ]:
samples[0].cam.plot() # still not going to work for me. sad. 

In [ ]:
type(samples[0].cam) # maybe this will work when i do it on my computer next week on the cpu lol

In [ ]:
samples[0].cam # Get gradcam working somehow plz

In [ ]:
samples

In [ ]:
model.generate_cams?